# Embedding + Bidirectional GRU

In [1]:
import os
import time
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


#### Reading data

In [2]:
dir_data = 'D:/Data_Master/Natural Language Processing/Project1/quora/'
train_df = pd.read_csv(dir_data+'train.csv')
dir_data = 'D:/Data_Master/Natural Language Processing/Project1/quora/'
test_df = pd.read_csv(dir_data+'test.csv')

#### Tokenizing the words for posterior embedding

In [3]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=123)

## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

#### Defining a basic model

In [4]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140160    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)  

#### Training

In [5]:
## Train the model 
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1044897 samples, validate on 261225 samples
Epoch 1/2


 104448/1044897 [=>............................] - ETA: 1:42:17 - loss: 0.6837 - acc: 0.83 - ETA: 1:11:34 - loss: 0.6676 - acc: 0.88 - ETA: 1:01:03 - loss: 0.6523 - acc: 0.90 - ETA: 54:57 - loss: 0.6360 - acc: 0.9180 - ETA: 51:59 - loss: 0.6200 - acc: 0.92 - ETA: 49:33 - loss: 0.6054 - acc: 0.92 - ETA: 48:16 - loss: 0.5883 - acc: 0.92 - ETA: 46:53 - loss: 0.5704 - acc: 0.92 - ETA: 46:03 - loss: 0.5541 - acc: 0.92 - ETA: 45:38 - loss: 0.5363 - acc: 0.92 - ETA: 44:52 - loss: 0.5179 - acc: 0.93 - ETA: 44:25 - loss: 0.5021 - acc: 0.93 - ETA: 43:52 - loss: 0.4866 - acc: 0.93 - ETA: 43:33 - loss: 0.4697 - acc: 0.93 - ETA: 43:06 - loss: 0.4560 - acc: 0.93 - ETA: 42:56 - loss: 0.4447 - acc: 0.93 - ETA: 42:44 - loss: 0.4339 - acc: 0.93 - ETA: 42:37 - loss: 0.4220 - acc: 0.93 - ETA: 42:23 - loss: 0.4112 - acc: 0.93 - ETA: 42:13 - loss: 0.4039 - acc: 0.93 - ETA: 42:05 - loss: 0.3919 - acc: 0.93 - ETA: 41:55 - loss: 0.3893 - acc: 0.93 - ETA: 41:49 - loss: 0.3837 - acc: 0.93 - ETA: 41:45 - loss: 0.

 208896/1044897 [====>.........................] - ETA: 38:48 - loss: 0.1838 - acc: 0.94 - ETA: 38:48 - loss: 0.1836 - acc: 0.94 - ETA: 38:47 - loss: 0.1833 - acc: 0.94 - ETA: 38:46 - loss: 0.1830 - acc: 0.94 - ETA: 38:45 - loss: 0.1826 - acc: 0.94 - ETA: 38:43 - loss: 0.1822 - acc: 0.94 - ETA: 38:42 - loss: 0.1819 - acc: 0.94 - ETA: 38:41 - loss: 0.1818 - acc: 0.94 - ETA: 38:40 - loss: 0.1813 - acc: 0.94 - ETA: 38:39 - loss: 0.1811 - acc: 0.94 - ETA: 38:38 - loss: 0.1809 - acc: 0.94 - ETA: 38:37 - loss: 0.1807 - acc: 0.94 - ETA: 38:36 - loss: 0.1803 - acc: 0.94 - ETA: 38:35 - loss: 0.1800 - acc: 0.94 - ETA: 38:33 - loss: 0.1797 - acc: 0.94 - ETA: 38:32 - loss: 0.1795 - acc: 0.94 - ETA: 38:31 - loss: 0.1790 - acc: 0.94 - ETA: 38:30 - loss: 0.1786 - acc: 0.94 - ETA: 38:29 - loss: 0.1787 - acc: 0.94 - ETA: 38:28 - loss: 0.1785 - acc: 0.94 - ETA: 38:28 - loss: 0.1782 - acc: 0.94 - ETA: 38:27 - loss: 0.1779 - acc: 0.94 - ETA: 38:30 - loss: 0.1777 - acc: 0.94 - ETA: 38:29 - loss: 0.1774 - a

 313344/1044897 [=======>......................] - ETA: 35:06 - loss: 0.1555 - acc: 0.94 - ETA: 35:05 - loss: 0.1554 - acc: 0.94 - ETA: 35:04 - loss: 0.1554 - acc: 0.94 - ETA: 35:03 - loss: 0.1552 - acc: 0.94 - ETA: 35:01 - loss: 0.1551 - acc: 0.94 - ETA: 35:00 - loss: 0.1549 - acc: 0.94 - ETA: 34:59 - loss: 0.1549 - acc: 0.94 - ETA: 34:57 - loss: 0.1548 - acc: 0.94 - ETA: 34:56 - loss: 0.1547 - acc: 0.94 - ETA: 34:55 - loss: 0.1547 - acc: 0.94 - ETA: 34:54 - loss: 0.1546 - acc: 0.94 - ETA: 34:52 - loss: 0.1545 - acc: 0.94 - ETA: 34:51 - loss: 0.1544 - acc: 0.94 - ETA: 34:50 - loss: 0.1543 - acc: 0.94 - ETA: 34:48 - loss: 0.1542 - acc: 0.94 - ETA: 34:47 - loss: 0.1541 - acc: 0.94 - ETA: 34:46 - loss: 0.1540 - acc: 0.94 - ETA: 34:44 - loss: 0.1539 - acc: 0.94 - ETA: 34:43 - loss: 0.1538 - acc: 0.94 - ETA: 34:41 - loss: 0.1536 - acc: 0.94 - ETA: 34:40 - loss: 0.1536 - acc: 0.94 - ETA: 34:39 - loss: 0.1535 - acc: 0.94 - ETA: 34:37 - loss: 0.1534 - acc: 0.94 - ETA: 34:36 - loss: 0.1533 - a

 417792/1044897 [==========>...................] - ETA: 30:39 - loss: 0.1434 - acc: 0.94 - ETA: 30:38 - loss: 0.1434 - acc: 0.94 - ETA: 30:37 - loss: 0.1434 - acc: 0.94 - ETA: 30:35 - loss: 0.1433 - acc: 0.94 - ETA: 30:34 - loss: 0.1432 - acc: 0.94 - ETA: 30:33 - loss: 0.1432 - acc: 0.94 - ETA: 30:32 - loss: 0.1431 - acc: 0.94 - ETA: 30:30 - loss: 0.1431 - acc: 0.94 - ETA: 30:29 - loss: 0.1430 - acc: 0.94 - ETA: 30:28 - loss: 0.1430 - acc: 0.94 - ETA: 30:27 - loss: 0.1429 - acc: 0.94 - ETA: 30:25 - loss: 0.1429 - acc: 0.94 - ETA: 30:24 - loss: 0.1429 - acc: 0.94 - ETA: 30:23 - loss: 0.1428 - acc: 0.94 - ETA: 30:22 - loss: 0.1428 - acc: 0.94 - ETA: 30:20 - loss: 0.1427 - acc: 0.94 - ETA: 30:19 - loss: 0.1427 - acc: 0.94 - ETA: 30:18 - loss: 0.1427 - acc: 0.94 - ETA: 30:16 - loss: 0.1427 - acc: 0.94 - ETA: 30:15 - loss: 0.1426 - acc: 0.94 - ETA: 30:14 - loss: 0.1426 - acc: 0.94 - ETA: 30:13 - loss: 0.1425 - acc: 0.94 - ETA: 30:11 - loss: 0.1425 - acc: 0.94 - ETA: 30:10 - loss: 0.1424 - a

...

 626688/1044897 [================>.............] - ETA: 25:00 - loss: 0.0966 - acc: 0.96 - ETA: 24:59 - loss: 0.0966 - acc: 0.96 - ETA: 24:57 - loss: 0.0966 - acc: 0.96 - ETA: 24:56 - loss: 0.0966 - acc: 0.96 - ETA: 24:55 - loss: 0.0966 - acc: 0.96 - ETA: 24:54 - loss: 0.0966 - acc: 0.96 - ETA: 24:53 - loss: 0.0967 - acc: 0.96 - ETA: 24:51 - loss: 0.0967 - acc: 0.96 - ETA: 24:50 - loss: 0.0967 - acc: 0.96 - ETA: 24:49 - loss: 0.0967 - acc: 0.96 - ETA: 24:48 - loss: 0.0967 - acc: 0.96 - ETA: 24:46 - loss: 0.0967 - acc: 0.96 - ETA: 24:45 - loss: 0.0967 - acc: 0.96 - ETA: 24:44 - loss: 0.0967 - acc: 0.96 - ETA: 24:43 - loss: 0.0967 - acc: 0.96 - ETA: 24:42 - loss: 0.0967 - acc: 0.96 - ETA: 24:40 - loss: 0.0967 - acc: 0.96 - ETA: 24:39 - loss: 0.0967 - acc: 0.96 - ETA: 24:38 - loss: 0.0967 - acc: 0.96 - ETA: 24:37 - loss: 0.0967 - acc: 0.96 - ETA: 24:35 - loss: 0.0967 - acc: 0.96 - ETA: 24:34 - loss: 0.0967 - acc: 0.96 - ETA: 24:33 - loss: 0.0967 - acc: 0.96 - ETA: 24:32 - loss: 0.0967 - a

 731136/1044897 [===================>..........] - ETA: 20:35 - loss: 0.0969 - acc: 0.96 - ETA: 20:34 - loss: 0.0969 - acc: 0.96 - ETA: 20:32 - loss: 0.0969 - acc: 0.96 - ETA: 20:31 - loss: 0.0969 - acc: 0.96 - ETA: 20:30 - loss: 0.0969 - acc: 0.96 - ETA: 20:28 - loss: 0.0969 - acc: 0.96 - ETA: 20:27 - loss: 0.0969 - acc: 0.96 - ETA: 20:26 - loss: 0.0969 - acc: 0.96 - ETA: 20:24 - loss: 0.0970 - acc: 0.96 - ETA: 20:23 - loss: 0.0969 - acc: 0.96 - ETA: 20:22 - loss: 0.0970 - acc: 0.96 - ETA: 20:20 - loss: 0.0970 - acc: 0.96 - ETA: 20:19 - loss: 0.0970 - acc: 0.96 - ETA: 20:17 - loss: 0.0970 - acc: 0.96 - ETA: 20:16 - loss: 0.0969 - acc: 0.96 - ETA: 20:15 - loss: 0.0969 - acc: 0.96 - ETA: 20:13 - loss: 0.0969 - acc: 0.96 - ETA: 20:12 - loss: 0.0969 - acc: 0.96 - ETA: 20:10 - loss: 0.0969 - acc: 0.96 - ETA: 20:09 - loss: 0.0969 - acc: 0.96 - ETA: 20:08 - loss: 0.0969 - acc: 0.96 - ETA: 20:06 - loss: 0.0969 - acc: 0.96 - ETA: 20:05 - loss: 0.0969 - acc: 0.96 - ETA: 20:03 - loss: 0.0970 - a

 835584/1044897 [======================>.......] - ETA: 15:46 - loss: 0.0970 - acc: 0.96 - ETA: 15:44 - loss: 0.0970 - acc: 0.96 - ETA: 15:43 - loss: 0.0970 - acc: 0.96 - ETA: 15:41 - loss: 0.0970 - acc: 0.96 - ETA: 15:40 - loss: 0.0970 - acc: 0.96 - ETA: 15:38 - loss: 0.0970 - acc: 0.96 - ETA: 15:37 - loss: 0.0970 - acc: 0.96 - ETA: 15:36 - loss: 0.0970 - acc: 0.96 - ETA: 15:34 - loss: 0.0970 - acc: 0.96 - ETA: 15:33 - loss: 0.0970 - acc: 0.96 - ETA: 15:31 - loss: 0.0970 - acc: 0.96 - ETA: 15:30 - loss: 0.0970 - acc: 0.96 - ETA: 15:28 - loss: 0.0970 - acc: 0.96 - ETA: 15:27 - loss: 0.0970 - acc: 0.96 - ETA: 15:25 - loss: 0.0970 - acc: 0.96 - ETA: 15:24 - loss: 0.0970 - acc: 0.96 - ETA: 15:22 - loss: 0.0970 - acc: 0.96 - ETA: 15:21 - loss: 0.0970 - acc: 0.96 - ETA: 15:19 - loss: 0.0970 - acc: 0.96 - ETA: 15:18 - loss: 0.0970 - acc: 0.96 - ETA: 15:16 - loss: 0.0970 - acc: 0.96 - ETA: 15:15 - loss: 0.0970 - acc: 0.96 - ETA: 15:14 - loss: 0.0970 - acc: 0.96 - ETA: 15:12 - loss: 0.0970 - a

 940032/1044897 [=========================>....] - ETA: 10:41 - loss: 0.0974 - acc: 0.96 - ETA: 10:39 - loss: 0.0974 - acc: 0.96 - ETA: 10:38 - loss: 0.0974 - acc: 0.96 - ETA: 10:36 - loss: 0.0974 - acc: 0.96 - ETA: 10:35 - loss: 0.0974 - acc: 0.96 - ETA: 10:33 - loss: 0.0974 - acc: 0.96 - ETA: 10:31 - loss: 0.0974 - acc: 0.96 - ETA: 10:30 - loss: 0.0974 - acc: 0.96 - ETA: 10:28 - loss: 0.0974 - acc: 0.96 - ETA: 10:27 - loss: 0.0974 - acc: 0.96 - ETA: 10:25 - loss: 0.0974 - acc: 0.96 - ETA: 10:24 - loss: 0.0974 - acc: 0.96 - ETA: 10:22 - loss: 0.0974 - acc: 0.96 - ETA: 10:21 - loss: 0.0974 - acc: 0.96 - ETA: 10:19 - loss: 0.0974 - acc: 0.96 - ETA: 10:18 - loss: 0.0974 - acc: 0.96 - ETA: 10:16 - loss: 0.0974 - acc: 0.96 - ETA: 10:15 - loss: 0.0974 - acc: 0.96 - ETA: 10:13 - loss: 0.0974 - acc: 0.96 - ETA: 10:11 - loss: 0.0974 - acc: 0.96 - ETA: 10:10 - loss: 0.0974 - acc: 0.96 - ETA: 10:08 - loss: 0.0974 - acc: 0.96 - ETA: 10:07 - loss: 0.0974 - acc: 0.96 - ETA: 10:05 - loss: 0.0974 - a

1044897/1044897 [==============================] - ETA: 5:24 - loss: 0.0976 - acc: 0.960 - ETA: 5:22 - loss: 0.0976 - acc: 0.960 - ETA: 5:20 - loss: 0.0976 - acc: 0.960 - ETA: 5:19 - loss: 0.0976 - acc: 0.960 - ETA: 5:17 - loss: 0.0976 - acc: 0.960 - ETA: 5:16 - loss: 0.0976 - acc: 0.960 - ETA: 5:14 - loss: 0.0976 - acc: 0.960 - ETA: 5:13 - loss: 0.0976 - acc: 0.960 - ETA: 5:11 - loss: 0.0976 - acc: 0.960 - ETA: 5:09 - loss: 0.0976 - acc: 0.960 - ETA: 5:08 - loss: 0.0976 - acc: 0.960 - ETA: 5:06 - loss: 0.0976 - acc: 0.960 - ETA: 5:05 - loss: 0.0976 - acc: 0.960 - ETA: 5:03 - loss: 0.0976 - acc: 0.960 - ETA: 5:02 - loss: 0.0976 - acc: 0.960 - ETA: 5:00 - loss: 0.0976 - acc: 0.960 - ETA: 4:58 - loss: 0.0976 - acc: 0.960 - ETA: 4:57 - loss: 0.0976 - acc: 0.960 - ETA: 4:55 - loss: 0.0976 - acc: 0.960 - ETA: 4:54 - loss: 0.0975 - acc: 0.960 - ETA: 4:52 - loss: 0.0975 - acc: 0.960 - ETA: 4:50 - loss: 0.0975 - acc: 0.960 - ETA: 4:49 - loss: 0.0975 - acc: 0.960 - ETA: 4:47 - loss: 0.0976 - ac

#### Fscore for validation with different thresholds

In [7]:
pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)

261225/261225 [==============================] - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - E

In [23]:
threshold = 0.27
print("F1 score at threshold {0} is {1}".format(threshold, metrics.f1_score(val_y, (pred_noemb_val_y>threshold).astype(int))))

F1 score at threshold 0.27 is 0.6471180106367688
